### Optimize user's portfolio using Efficient Frontier

In [ ]:
from pandas_datareader import data as web
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
from datetime import datetime
import seaborn as sns
import cvxopt
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import DiscreteAllocation, get_latest_prices
from pypfopt import risk_models
from pypfopt import expected_returns
plt.style.use('ggplot')
plt.style.use('seaborn-poster')
sns.set_palette('muted')

In [ ]:
def check_weights(x):
    if np.sum(x) != 1:
        print('Current weights total:', np.sum(x))
        print('Please make sure your weights add up to 1')
        return None
    return x
def check_dates_length(x,y):
    if ((len(x)) != len((y))) & (len((y)) != 1):
        print('Please enter the start dates for the tickers in the format "YYYY-MM-DD" seperated by commas')
        print('Please make sure there are the same amount of dates as tickers, or a single date.')
        return None
    return x, y
def check_weights_length(x,y):
    if len(x) != len(y):
        print('Please enter the same number of weights as tickers')
        return None
    return x, y

In [ ]:
def create_df(tickers, start_dates, weights, end_date = datetime.today().strftime('%Y-%m-%d'),api='av-daily-adjusted',
         params = 'adjusted close'):
    if type(tickers) != list:
        tickers = tickers.split()
    if type(start_dates) != list:
        start_dates = start_dates.split()
    check_dates_length(tickers,start_dates)
    check_weights_length(tickers,weights)
    check_weights(weights)
    df = pd.DataFrame()
    if len(start_dates) == 1:
        for i in tickers:
            df[i] = web.DataReader(i, api, start = start_dates[0], end = end_date, api_key= '58O6Y312RH95S2QY')[params]
    else:
        for i,j in zip(tickers,start_dates):
            df[i] = web.DataReader(i, api, start = j, end = end_date, api_key= '58O6Y312RH95S2QY')[params]
    return df


In [ ]:
def time_series_graph(df):
    df1 = df.copy()
    df1.reset_index(inplace=True)
    df1.rename(columns={'index':'Date'},inplace=True)
    df1['Date'] = pd.to_datetime(df1['Date'], format = '%Y-%m-%d')
    for i in df1.columns[1:].values:
        ax = sns.lineplot(data=df1, x = df1['Date'], y = df1[i], label = i)
        ax.xaxis.set_minor_locator(md.MonthLocator(interval = 1))
        plt.xlabel('Date',fontsize=18)
        plt.ylabel('Adj. Price in USD', fontsize=18)
        plt.title('Adjusted Close Price History')
        plt.gcf().autofmt_xdate()
        plt.tight_layout()

In [ ]:
def get_covariance_matrix(df):
    returns = df.pct_change()
    cov_matrix_annual = returns.cov() * 252
    return returns, cov_matrix_annual
def get_vvr(df,weights,verbose=True):
    returns,covariance = get_covariance_matrix(df)
    variance = np.dot(weights.T, np.dot(covariance, weights))
    volatility = np.sqrt(variance)
    simple_annual_returns = np.sum(returns.mean() * weights) * 252
    var_percent = round(variance * 100, 2)
    vol_percent = round(volatility * 100, 2)
    annual_returns_percent = round(simple_annual_returns * 100, 2)
    if verbose == True:
        print('Expected Annual Returns: ' + str(annual_returns_percent) + '%')
        print('Annual Volatiltiy: ' + str(vol_percent) + '%')
        print('Annual Variance: ' + str(var_percent) + '%')


In [ ]:
def optimize(df,total_portfolio_value):
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)
    ef = EfficientFrontier(mu,S)
    w = ef.max_sharpe()
    clean_w = ef.clean_weights()
    print('Sharpe Performance')
    print(ef.portfolio_performance(verbose=1))
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(clean_w, latest_prices, total_portfolio_value)
    allocation, leftover = da.lp_portfolio()
    print('Optimized Portfolio')
    print('Optimized Allocation: ', allocation)
    print('Funds Remaining: ${:.2f}'.format(leftover))


In [ ]:
def main(tickers,start_dates,weights,tpv,graph=False,verbose=True):
    df = create_df(tickers,start_dates,weights)
    if graph == True:
        time_series_graph(df)
    get_vvr(df,weights,verbose)
    optimize(df,tpv)

In [ ]:
input_tickers = input('Enter ticker symbols seperated by a space:')
tickers = input_tickers.upper().split()
input_weights = input('Enter weight as decimal percentage matching their tickers seperated by a space:')
weights = np.array([float(i) for i in input_weights.split()])
input_start_dates = input('Enter start date(s) depeding on if you have multiple or one, format YYYY-MM-DD: ')
start_dates = input_start_dates.split()
value= int(input('Enter total protfolio value in USD:'))

In [ ]:
main(tickers,start_dates,weights,value,graph=True)